# **COVID-19 Data Study Notebook**

## Objectives

* Fetch Data from Kaggle and save as raw file and unzip.
* Inspect the data and save it under inputs/datasets/raw
* Inspect the data and save it under outputs/datasets/collection

## Inputs

* Kaggle JSON file - the authentication token.

## Outputs

* Generate Dataset: outputs/datasets/collection/trimmed_covid_dataset.csv

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/milestone-covid-19-study/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/milestone-covid-19-study'

---

# Load Data

In [5]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/covid-19-dataset.csv")
    )
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL
0,2,12,1,1,9999-99-99,2.0,27.0,2.0,2,2,2,2,2,2,2,2,2,2,7
1,2,12,1,1,9999-99-99,2.0,23.0,2.0,2,2,2,2,2,2,2,2,2,2,7
2,1,12,1,2,9999-99-99,2.0,22.0,2.0,2,2,2,2,2,2,2,2,2,2,3
3,1,12,1,1,9999-99-99,2.0,37.0,2.0,2,2,2,2,2,2,2,2,2,2,7
4,2,12,1,1,9999-99-99,2.0,29.0,2.0,2,2,2,2,1,2,2,2,2,1,7


# Data Exploration

We are interested to get more familiar with the dataset, check variable type and distribution, missing levels and what these variables mean in a business context

# Correlaton Study

code

## Conclusions and Next Steps

The correlations and plots interpretation converge.

- A churned customer typically has a month-to-month contract.
- A churned customer typically has fiber optic.
- A churned customer typically doesn't have tech support.
- A churned customer doesn't have online security.
- A churned customer typically has low tenure levels